In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [3]:
def get_url(url):
    try:
        with closing(get(url, stream=True)) as response:
            if is_good(response):
                return response.content
            else:
                return None

    except RequestException as e:
        print('Error during requests to {0} : {1}'.format(url, str(e)))
        return None
def is_good(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


In [65]:
showscore = 'https://www.show-score.com/categories/top-scoring-on-broadway'
showscore2 = 'https://www.show-score.com/shows/all?opened=yes'
raw_html = get_url(showscore)
print(len(raw_html))

131636


In [71]:
import re
html = BeautifulSoup(raw_html, 'html.parser')

# get name
musical_dict = {}
for idx, mess in enumerate(html.select('.show-tile__name')):
#     print(mess)
    messString = str(mess)
    pattern = re.compile('\>(.*?)\n')
    match = pattern.search(messString)
#     if match != None:
    name = match.group(1)
#     print("name: ", name)
    musical_dict[idx] = [name]
#     else:
#         pattern = re.compile('\>(.*?)\.')
#         match = pattern.search(messString)
#         name = match.group(1)
#         print("name: ", name)
    
# get description
for idx,p in enumerate(html.select('p')):
#     print("description: ",p.text)
    if idx in musical_dict:
        musical_dict[idx].append(p.text)
    
    
# get score
for idx, mess in enumerate(html.select('.positive.show-tile__score')):
#     print(mess)
    messString = str(mess)
    pattern = re.compile('<span class="positive show-tile__score"\>\n(\d+)')
    match = pattern.search(messString)
    score = match.group(1)
#     print("score: ", score)
    musical_dict[idx].append(score)
print(musical_dict)

{0: ['Hamilton (NYC)', "Lin-Manuel Miranda's hip-hop infused tuner about Alexander Hamilton began at The Public Theater to rave reviews, then transferred to Broadway where it won eleven 2016 Tony Awards including Best Musical. ", '97'], 1: ['Wicked (NYC)', 'Now in its fourteenth year on Broadway, this all-ages spectacle follows the untold true story of the Witches of Oz.', '93'], 2: ['Dear Evan Hansen (NYC)', "'Dear Evan Hansen' is about how we share, like, and friend, love, feel, and survive in the modern world. Winner of six 2017 Tony Awards, including Best Musical.", '93'], 3: ['The Lion King (NYC)', "In Julie Taymor's Tony Award winning retelling of Disney's animated hit, puppets and live actors weave a dreamscape of jungle characters, creating the story of young Simba's rise to the throne.", '91'], 4: ['Come From Away (NYC)', 'This Broadway sleeper hit musical tells the heartwarming true story a tiny Canadian town that cared for thousands of grounded travelers in the wake of the 9

In [75]:
import csv
with open('showscore.csv','w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(['Name','Score', 'Description'])
    for idx, musicals in musical_dict.items():
        writer.writerow([musicals[0],musicals[2], musicals[1]])